In [1]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [4]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

In [7]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [8]:
loss_fn = nn.CrossEntropyLoss()

In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [18]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss:{test_loss:>8f} \n")

In [17]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
-----------------------------
loss: 1.289009 [    0/60000]
loss: 1.427784 [ 6400/60000]
loss: 1.255570 [12800/60000]
loss: 1.662971 [19200/60000]
loss: 1.344890 [25600/60000]
loss: 1.431935 [32000/60000]
loss: 1.449762 [38400/60000]
loss: 1.287106 [44800/60000]
loss: 1.411852 [51200/60000]
loss: 1.658062 [57600/60000]
Test Error: 
 Accuracy: 53.1%, Avg loss:0.022533 

Epoch 2
-----------------------------
loss: 1.261438 [    0/60000]
loss: 1.401375 [ 6400/60000]
loss: 1.227535 [12800/60000]
loss: 1.646407 [19200/60000]
loss: 1.332305 [25600/60000]
loss: 1.413291 [32000/60000]
loss: 1.432277 [38400/60000]
loss: 1.269274 [44800/60000]
loss: 1.393638 [51200/60000]
loss: 1.642296 [57600/60000]
Test Error: 
 Accuracy: 54.0%, Avg loss:0.022238 

Epoch 3
-----------------------------
loss: 1.235470 [    0/60000]
loss: 1.377498 [ 6400/60000]
loss: 1.201715 [12800/60000]
loss: 1.631931 [19200/60000]
loss: 1.320752 [25600/60000]
loss: 1.396435 [32000/60000]
loss: 1.416029 [38400/60000]
l

In [16]:
torch.save(model.state_dict(), "data/model.pth")

print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
